# Model Training

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
import gc
import time
import itertools
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, log_loss, roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns
import gzip
import torch
from deepctr_torch.models import DeepFM,DCN
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

In [2]:
np.random.seed(42)


## Data Preprocessing and Feature Engineering

In [3]:
#load data
train = pd.read_csv("./data/my_train.csv", header=0, dtype=str)
test_df = pd.read_csv("./data/my_test.csv", header=0, dtype=str)

In [4]:
len(test_df)

3000000

In [5]:
test_df['data_label'] = 'test'
train['data_label'] = 'train'
train_test = pd.concat([test_df,train])
data1 = train_test

In [6]:
data1['hour'] = pd.to_datetime(data1['hour'], format = '%y%m%d%H')
data1['click'] = pd.to_numeric(data1['click'], errors='coerce')
data1['dt'] = data1['hour'].apply(lambda x:str(x)[:10])
device_id_sum = data1.groupby(['dt','device_id']).click.sum().reset_index(name='id_sum')
device_id_cnt = data1.groupby(['dt','device_id']).click.count().reset_index(name='id_cnt')
cnt_sum = pd.merge(device_id_cnt,device_id_sum,on=(['dt','device_id']),how='left')
cnt_sum['device_ctr'] = cnt_sum['id_sum']/cnt_sum['id_cnt']
cnt_sum = cnt_sum.sort_values(['device_id','dt'])
cnt_sum['device_ctr'] = cnt_sum['id_sum']/cnt_sum['id_cnt']
cnt_sum = cnt_sum.sort_values(['device_id','dt'])
cnt_sum['t_1']=cnt_sum.groupby(['device_id'])['device_ctr'].shift(1)
cnt_sum.groupby('device_id')['dt'].count().reset_index().sort_values('dt',ascending=False)
data2 = pd.merge(data1,cnt_sum,on=(['dt','device_id']),how='left')

In [7]:
data1 = data2

In [8]:
data1['hour_of_day'] = data1['hour'].dt.hour
data1['weekday'] = (data1['hour'].dt.dayofweek)

In [9]:
obj_cols = ['site_id',
 'site_domain',
 'site_category',
 'app_id',
 'app_domain',
 'app_category',
 'device_id',
 'device_ip',
 'device_model','device_type','device_conn_type'
,'hour_of_day','weekday','banner_pos']
c_cols = ['C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21','C1']

In [10]:
# sparse特征列
sparse_features = obj_cols+['hour_of_day','weekday']+ c_cols


In [11]:
# Padding
for f in ['device_type','device_conn_type','banner_pos'
          ,'C14','C15','C16','C17','C18', 'C19', 'C20', 'C21', 'C1']:
    data1[f] = data1[f].astype(str)

In [12]:
for f in sparse_features:
    data1[f] = data1[f].fillna('-1', )

In [13]:
# 对sparse onehot
for feat in sparse_features:
    lbe = LabelEncoder()
    data1[feat] = lbe.fit_transform(data1[feat])

In [14]:
# Continuous Features
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
dense_features = ['id_cnt',
 'id_sum',
 'device_ctr',
 't_1']
for f in dense_features:
    data1[f] = data1[f].fillna(0, )

mms = MinMaxScaler(feature_range=(0, 1))
data1[dense_features] = mms.fit_transform(data1[dense_features])


In [15]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data1[feat].nunique(),embedding_dim=4 )
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]
# fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data1[feat].nunique(),embedding_dim=4 )
#                        for i,feat in enumerate(sparse_features)]


In [16]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)


In [17]:
train = data1[data1['data_label']=='train']
train_1 = train[:27000]
train_2 = train[30000:360000]
test_old = data1[data1['data_label']=='test']


In [18]:
test_1 = train[27000:30000]
test_1_model_input = {name:test_1[name] for name in feature_names}

In [19]:
train_1_model_input = {name:train_1[name] for name in feature_names}
train_2_model_input = {name:train_2[name] for name in feature_names}

test_model_input = {name:test_old[name] for name in feature_names}

In [20]:
import numpy as np

def binary_cross_entropy(y_true, y_pred):
    """
    Compute Binary Cross-Entropy Loss.

    Args:
        y_true (numpy.ndarray): True binary labels (shape: [n_samples]).
        y_pred (numpy.ndarray): Predicted probabilities (shape: [n_samples]).

    Returns:
        float: Binary Cross-Entropy loss.
    """
    # Clip predictions to avoid log(0)
    y_pred = np.clip(y_pred, 1e-12, 1. - 1e-12)
    # Compute BCE
    bce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return bce


In [21]:
from deepctr_torch.models import DeepFM,AFM,xDeepFM,AutoInt,DCN,DIFM
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'
# Set random seed
torch.manual_seed(42)
np.random.seed(42)

In [22]:
model = DCN(linear_feature_columns,dnn_feature_columns,task='binary',device=device)
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model.fit(train_1_model_input,train_1['click'].values,batch_size=128,epochs=1,verbose=2,validation_split=0.1)
train_2['pred']= model.predict(train_2_model_input, batch_size=128)
collected_train = train_2.groupby(train_2.index // 10).apply(lambda x: x.loc[x["pred"].idxmax()])
test_old['pred_1']= model.predict(test_model_input, batch_size=128)
        

cpu
Train on 24300 samples, validate on 2700 samples, 190 steps per epoch
Epoch 1/1
5s - loss:  0.4312 - binary_crossentropy:  0.4312 - val_binary_crossentropy:  0.4038


In [23]:
model_best = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device)
model_best.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model_best.fit(train_1_model_input,train_1['click'].values,batch_size=128,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_best']= model_best.predict(test_model_input, batch_size=128)


cpu
Train on 24300 samples, validate on 2700 samples, 190 steps per epoch
Epoch 1/1
5s - loss:  0.4268 - binary_crossentropy:  0.4267 - val_binary_crossentropy:  0.3968


In [24]:
validation = collected_train[:3000]
validation_model_input = {name:validation[name] for name in feature_names}
new_train=collected_train[3000:]
new_train_model_input = {name:new_train[name] for name in feature_names}


In [35]:
collected_train['click'].to_csv("offline_data.csv")

In [25]:
model2 = DIFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device)
model2.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model2.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_2']= model2.predict(test_model_input, batch_size=128)

model3 = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device)
model3.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model3.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_3']= model3.predict(test_model_input, batch_size=128)

model4 = DCN(linear_feature_columns,dnn_feature_columns,task='binary',device=device)
model4.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model4.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_4']= model4.predict(test_model_input, batch_size=128)

model5 = DIFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device, dnn_hidden_units=(256, 128, 64),)
model5.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model5.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_5']= model5.predict(test_model_input, batch_size=128)

model6 = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device, dnn_hidden_units=(128, 128, 64))
model6.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model6.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_6']= model6.predict(test_model_input, batch_size=128)

model7 = DCN(linear_feature_columns,dnn_feature_columns,task='binary',device=device, dnn_hidden_units=(256, 128, 64),)
model7.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model7.fit(new_train_model_input,new_train['click'].values,batch_size=64,epochs=1,verbose=2,validation_split=0.1)
test_old['pred_7']= model7.predict(test_model_input, batch_size=128)

cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
12s - loss:  0.5953 - binary_crossentropy:  0.5953 - val_binary_crossentropy:  0.5768
cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
10s - loss:  0.5909 - binary_crossentropy:  0.5909 - val_binary_crossentropy:  0.5671
cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
10s - loss:  0.5952 - binary_crossentropy:  0.5951 - val_binary_crossentropy:  0.5777
cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
10s - loss:  0.5969 - binary_crossentropy:  0.5968 - val_binary_crossentropy:  0.5768
cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
9s - loss:  0.5936 - binary_crossentropy:  0.5936 - val_binary_crossentropy:  0.5702
cpu
Train on 27000 samples, validate on 3000 samples, 422 steps per epoch
Epoch 1/1
10s - loss:  0.5965 - binary_crossentropy:  0.5965 - val_binary_cro

In [26]:
import torch.nn as nn

# Define Binary Cross-Entropy loss function
criterion = nn.BCELoss()


In [27]:
test_1["pred_1"]= model.predict(test_1_model_input, batch_size=128)
test_1["pred_2"]= model2.predict(test_1_model_input, batch_size=128)
test_1["pred_3"]= model3.predict(test_1_model_input, batch_size=128)
test_1["pred_4"]= model4.predict(test_1_model_input, batch_size=128)
test_1["pred_5"]= model5.predict(test_1_model_input, batch_size=128)
test_1["pred_6"]= model6.predict(test_1_model_input, batch_size=128)
test_1["pred_7"]= model7.predict(test_1_model_input, batch_size=128)

test_1["pred_best"]= model_best.predict(test_1_model_input, batch_size=128)

for num_model in ["pred_1","pred_2","pred_3","pred_4","pred_5","pred_6","pred_7","pred_best"]:
    print(num_model, ", binary cross entropy loss: ", criterion(torch.tensor(test_1[num_model].values, dtype=torch.float32)
                                                                ,torch.tensor(test_1["click"].values, dtype=torch.float32)).item())

pred_1 , binary cross entropy loss:  0.39470598101615906
pred_2 , binary cross entropy loss:  0.8969165682792664
pred_3 , binary cross entropy loss:  0.6406513452529907
pred_4 , binary cross entropy loss:  0.7149401307106018
pred_5 , binary cross entropy loss:  1.0635292530059814
pred_6 , binary cross entropy loss:  0.6902387738227844
pred_7 , binary cross entropy loss:  0.7526766657829285
pred_best , binary cross entropy loss:  0.3880247473716736


In [28]:
validation["pred_1"]= model.predict(validation_model_input, batch_size=128)
validation["pred_2"]= model2.predict(validation_model_input, batch_size=128)
validation["pred_3"]= model3.predict(validation_model_input, batch_size=128)
validation["pred_4"]= model4.predict(validation_model_input, batch_size=128)
validation["pred_5"]= model5.predict(validation_model_input, batch_size=128)
validation["pred_6"]= model6.predict(validation_model_input, batch_size=128)
validation["pred_7"]= model7.predict(validation_model_input, batch_size=128)

validation["pred_best"]= model_best.predict(validation_model_input, batch_size=128)

for num_model in ["pred_1","pred_2","pred_3","pred_4","pred_5","pred_6","pred_7","pred_best"]:
    print(num_model, ", binary cross entropy loss: ", criterion(torch.tensor(validation[num_model].values, dtype=torch.float32)
                                                                ,torch.tensor(validation["click"].values, dtype=torch.float32)).item())

pred_1 , binary cross entropy loss:  0.6664384603500366
pred_2 , binary cross entropy loss:  0.5767185688018799
pred_3 , binary cross entropy loss:  0.5668131709098816
pred_4 , binary cross entropy loss:  0.5762074589729309
pred_5 , binary cross entropy loss:  0.5760177969932556
pred_6 , binary cross entropy loss:  0.5721731185913086
pred_7 , binary cross entropy loss:  0.5796493291854858
pred_best , binary cross entropy loss:  0.6481015682220459


In [29]:
def compute_reward_table(test_old):
    reward1 = test_old.groupby(test_old.index // 10).apply(lambda x: x.loc[x["pred_1"].idxmax()])
    for num_model in ["pred_2","pred_3","pred_4","pred_5","pred_6","pred_7","pred_best"]:
        reward1[num_model] = test_old.groupby(test_old.index // 10).apply(lambda x: x.loc[x[num_model].idxmax()])
    

In [ ]:
reward1 = test_old.groupby(test_old.index // 10).apply(lambda x: x.loc[x["pred_1"].idxmax()])[["click"]]
for num_model in ["pred_2","pred_3","pred_4","pred_5","pred_6","pred_7","pred_best"]:
    rw =test_old.groupby(test_old.index // 10).apply(lambda x: x.loc[x[num_model].idxmax()])[["click"]]
    reward1[num_model] = rw["click"].values

In [ ]:
reward1.rename(columns={'click': 'pred_1'}, inplace=True)


In [36]:
np.mean(reward1,axis=0)

pred_1       0.429330
pred_2       0.167433
pred_3       0.260790
pred_4       0.255587
pred_5       0.165907
pred_6       0.242280
pred_7       0.249797
pred_best    0.451547
dtype: float64

In [ ]:
reward1.to_csv("reward_table_bis.csv")

In [32]:
len(reward1)

300000

In [37]:
np.mean(collected_train['click'])

0.4306969696969697